<a href="https://colab.research.google.com/github/AllyHyeseongKim/CAU11934_MachineLarning/blob/master/assignment/06/assignment06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>